## Конвертация YOLO-датасета в COCO фомат

In [10]:
from globox import AnnotationSet, Annotation, BoundingBox
from pathlib import Path, PosixPath
from PIL import Image
import numpy as np
from tqdm import tqdm

In [26]:
labels_path = Path("/mnt/work_share/DWH/DataSets/Detection/10/0/yolo/labels/test/")
images_path = Path("/mnt/work_share/DWH/DataSets/Detection/10/0/yolo/images/test/")
labels = Path("/home/rybin/Dev/assets/nlmk11/obj.names").read_text().splitlines()

In [ ]:
def find_open_img(imgs_path:PosixPath, name):
    from PIL import Image
    impath = imgs_path / name
    for ext in [".jpg", ".JPG", ".jpeg", ".JPEG", ".png", ".PNG"]:
        impath_ = impath.with_suffix(ext)
        if impath_.exists():
            try:
                return Image.open(impath_), impath_
            except:
                print("error open", impath_)
                pass
    return None, impath

annset = AnnotationSet()
lab_names = sorted([p for p in labels_path.glob("*.txt")])
for i, lab_path in enumerate(tqdm(lab_names)):
    img, img_path = find_open_img(images_path, lab_path.stem)
    if img is None:
        print("missing", img_path)
    else:
        w, h = img.size
        ann = Annotation(img_path.name, (w, h))
        ann_txt = Path(lab_path).read_text()
        for line in ann_txt.splitlines():
            cls_id, *bbox_str = line.split()
            x0, y0, bw, bh = map(float, bbox_str)
            bbox = BoundingBox(label=labels[int(cls_id)], 
                            xmin=w*(x0 - bw/2),
                            ymin=h*(y0 - bh/2),
                            xmax=w*(x0 + bw/2),
                            ymax=h*(y0 + bh/2),
                            confidence=1)
            print(bbox.is_ground_truth)
            ann.add(bbox)
    annset.add(ann, override=True)

In [7]:
print(annset.show_stats())

            Database Stats             
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ Label              ┃ Images ┃ Boxes ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ <empty_image>      │     58 │     0 │
│ bicycle            │    328 │   979 │
│ car                │   1144 │  4163 │
│ dead_stop          │    119 │   123 │
│ derail             │    134 │   205 │
│ hand_shifter_left  │    307 │   465 │
│ hand_shifter_right │    284 │   450 │
│ letter             │   1401 │  1947 │
│ maneuver           │     34 │    35 │
│ milestone          │    396 │   397 │
│ motorcycle         │    190 │   398 │
│ person             │   1296 │  5675 │
│ pillar             │   3263 │  6109 │
│ sign               │     54 │    63 │
│ sound_signal       │    241 │   318 │
│ tflight            │   2502 │  6651 │
│ train              │   2471 │  5300 │
│ worker             │    505 │   749 │
├────────────────────┼────────┼───────┤
│ Total              │   7186 │ 34027 │
└────────────────────┴────────┴───────┘

None


In [8]:
annset._id_to_imageid = {i:img for i, img in enumerate(annset.image_ids)}
annset._id_to_label = {i:lab for i, lab in enumerate(labels)}

In [9]:
import json
json.dump(annset.to_coco(), open("/home/rybin/Dev/assets/nlmk11/annotation.json", "w"))

## Вычисление метрик

In [2]:
from globox import AnnotationSet
annset_gt = AnnotationSet.from_coco("/home/rybin/Dev/assets/nlmk11/annotation.json")
annset_pr = AnnotationSet.from_coco("/home/rybin/Dev/yolov7/runs/detect/yolov7d6-nlmk11-3/annotation.json")

In [21]:
from globox import COCOEvaluator

evaluator = COCOEvaluator(ground_truths=annset_gt, predictions=annset_pr)
evaluator.show_summary()

                                                  COCO Evaluation                                                  
┏━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃        ┃     AP ┃        ┃        ┃        ┃        ┃        ┃        ┃       ┃        ┃       ┃        ┃       ┃
┃ Label  ┃  50:95 ┃  AP 50 ┃  AP 75 ┃   AP S ┃   AP M ┃   AP L ┃   AR 1 ┃ AR 10 ┃ AR 100 ┃  AR S ┃   AR M ┃  AR L ┃
┡━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ bicyc… │ 38.31% │ 58.77% │ 42.35% │ 21.01% │ 29.30% │ 47.54% │ 22.26% │ 37.9… │ 38.31% │ 28.0… │ 34.58% │ 50.2… │
│ car    │ 50.80% │ 68.00% │ 58.07% │ 39.04% │ 44.86% │ 61.32% │ 21.04% │ 50.5… │ 50.80% │ 44.8… │ 49.36% │ 64.1… │
│ dead_… │ 58.12% │ 90.49% │ 65.60% │ 54.83% │ 77.21% │   nan% │ 56.27% │ 58.1… │ 58.12% │ 60.6… │ 80.00% │  nan% │
│ derail │ 56.94% │ 86.65% │ 67.87% │ 36.07% │ 69.94% │ 68.78% │ 43.64% │ 56.9… │ 56.94% │ 39.4… │ 74.50% │ 72.4… │
│ hand_… │ 73.57% │ 95.88% │ 88.28% │ 49.97% │ 69.87% │ 82.71% │ 51.23% │ 73.5… │ 73.57% │ 59.1… │ 74.21% │ 85.2… │
│ hand_… │ 73.80% │ 96.30% │ 88.37% │ 41.17% │ 68.88% │ 80.77% │ 48.95% │ 73.8… │ 73.80% │ 63.8… │ 74.57% │ 84.0… │
│ letter │ 50.39% │ 87.33% │ 54.07% │ 42.99% │ 64.65% │ 72.12% │ 38.48% │ 50.3… │ 50.39% │ 49.8… │ 70.77% │ 77.5… │
│ maneu… │ 71.53% │ 91.86% │ 77.10% │ 67.27% │ 75.45% │ 100.0… │ 70.12% │ 71.5… │ 71.53% │ 69.4… │ 78.67% │ 100.… │
│ miles… │ 59.48% │ 93.11% │ 66.76% │ 41.89% │ 60.76% │ 65.61% │ 59.26% │ 59.4… │ 59.48% │ 51.2… │ 66.89% │ 71.5… │
│ motor… │ 42.08% │ 61.44% │ 48.50% │ 18.21% │ 28.92% │ 50.04% │ 27.88% │ 41.9… │ 42.08% │ 22.8… │ 33.64% │ 53.5… │
│ person │ 41.47% │ 64.05% │ 46.90% │ 27.10% │ 37.65% │ 51.92% │ 15.80% │ 39.2… │ 41.47% │ 34.7… │ 42.81% │ 56.3… │
│ pillar │ 51.60% │ 91.10% │ 52.54% │ 46.92% │ 65.98% │ 65.21% │ 31.84% │ 51.5… │ 51.60% │ 53.7… │ 72.95% │ 72.5… │
│ sign   │ 52.46% │ 79.15% │ 62.59% │ 47.24% │ 68.06% │ 45.45% │ 49.83% │ 52.4… │ 52.46% │ 53.0… │ 70.56% │ 45.0… │
│ sound… │ 73.70% │ 96.52% │ 90.24% │ 67.81% │ 77.51% │ 83.80% │ 57.09% │ 73.7… │ 73.70% │ 73.2… │ 81.69% │ 87.9… │
│ tflig… │ 53.91% │ 86.58% │ 58.72% │ 45.38% │ 66.58% │ 69.33% │ 23.68% │ 53.2… │ 53.91% │ 52.1… │ 71.34% │ 73.5… │
│ train  │ 51.04% │ 70.57% │ 56.21% │ 42.13% │ 43.34% │ 53.49% │ 32.32% │ 51.0… │ 51.04% │ 52.3… │ 50.64% │ 58.9… │
│ worker │ 61.77% │ 85.75% │ 70.84% │ 38.56% │ 55.23% │ 72.46% │ 47.26% │ 61.7… │ 61.77% │ 48.4… │ 61.44% │ 76.3… │
├────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼───────┼────────┼───────┼────────┼───────┤
│ Total  │ 56.53% │ 82.56% │ 64.41% │ 42.80% │ 59.07% │ 66.91% │ 43.66% │ 60.7… │ 61.06% │ 50.4… │ 64.04% │ 70.5… │
└────────┴────────┴────────┴────────┴────────┴────────┴────────┴────────┴───────┴────────┴───────┴────────┴───────┘